# 🔄 Dynamic Programming for Sequence Optimization

**Welcome, St. Mark!** In this notebook, we'll explore dynamic programming - the art of breaking complex problems into simpler subproblems. Think of it as solving a puzzle by first solving smaller pieces.

We'll implement:
1. **Edit Distance** - Measuring sequence similarity for text processing
2. **Knapsack Problem** - Resource allocation optimization
3. **Longest Common Subsequence** - Finding patterns in sequences

By the end, you'll understand how DP enables efficient ML algorithms for sequence data.

## The Big Picture

Dynamic programming solves optimization problems by:
- **Breaking down**: Complex problems into overlapping subproblems
- **Memoization**: Storing solutions to avoid recomputation
- **Optimal substructure**: Building solutions from optimal subsolutions

**Key Question:** How can we efficiently solve sequence optimization problems that arise in ML applications?

In [ ]:
import numpy as np
from collections import defaultdict
import matplotlib.pyplot as plt
from functools import lru_cache
import time

# Nigerian healthcare context: Text processing for medical records
print("Setting up dynamic programming for sequence optimization...")
print("Applications: Text similarity, resource allocation, pattern matching")

**Cell Analysis:** We've imported libraries for efficient DP implementations.

- **numpy**: Matrix operations for DP tables
- **functools.lru_cache**: Automatic memoization for recursive solutions
- **collections**: Efficient data structures
- **matplotlib**: Visualization of DP tables and solutions

**Healthcare Context:** Nigerian medical systems need efficient text processing for patient records, drug matching, and resource optimization.

**Reflection Question:** How would DP help match patient names across different Nigerian hospital databases?

## Method 1: Edit Distance - Sequence Similarity for ML

**Healthcare Analogy:** Measuring how similar two DNA sequences are, or how different two patient diagnoses appear.

**ML Application:** Text similarity scoring for duplicate detection, spell checking, and sequence alignment.

**Algorithm:** Dynamic programming table where each cell represents edit distance between substrings.

In [ ]:
def edit_distance(str1, str2):
    """
    Levenshtein distance: Minimum operations to transform str1 into str2.
    
    Operations: Insert, Delete, Substitute (each costs 1)
    
    Parameters:
    str1, str2: Input strings to compare
    
    Returns:
    distance: Minimum edit distance
    dp_table: DP table for visualization
    """
    m, n = len(str1), len(str2)
    
    # Initialize DP table
    dp = np.zeros((m+1, n+1), dtype=int)
    
    # Base cases: empty string transformations
    for i in range(m+1):
        dp[i][0] = i  # Delete all characters from str1
    for j in range(n+1):
        dp[0][j] = j  # Insert all characters into str1
    
    # Fill DP table
    for i in range(1, m+1):
        for j in range(1, n+1):
            # If characters match, no operation needed
            if str1[i-1] == str2[j-1]:
                dp[i][j] = dp[i-1][j-1]
            else:
                # Minimum of: substitute, delete, insert
                dp[i][j] = min(
                    dp[i-1][j-1] + 1,  # Substitute
                    dp[i-1][j] + 1,     # Delete
                    dp[i][j-1] + 1      # Insert
                )
    
    return dp[m][n], dp

def visualize_edit_distance(str1, str2, dp_table):
    """Visualize the edit distance DP table"""
    fig, ax = plt.subplots(figsize=(10, 8))
    
    # Display table as heatmap
    im = ax.imshow(dp_table, cmap='Blues')
    
    # Add text annotations
    for i in range(len(str1)+1):
        for j in range(len(str2)+1):
            text = ax.text(j, i, dp_table[i, j],
                         ha="center", va="center", color="black", fontsize=12)
    
    # Labels
    ax.set_xticks(np.arange(len(str2)+1))
    ax.set_yticks(np.arange(len(str1)+1))
    ax.set_xticklabels([''] + list(str2))
    ax.set_yticklabels([''] + list(str1))
    
    ax.set_xlabel(f'String 2: "{str2}"')
    ax.set_ylabel(f'String 1: "{str1}"')
    ax.set_title('Edit Distance DP Table\n(Darker = Higher Cost)')
    
    plt.colorbar(im)
    plt.tight_layout()
    plt.show()

**Algorithm Analysis:**

- **DP Table**: Each cell dp[i][j] represents edit distance between first i chars of str1 and first j chars of str2
- **Base Cases**: Transforming to/from empty strings requires insertions/deletions
- **Optimal Substructure**: Solution builds from smaller substring solutions
- **Time Complexity:** O(m*n) where m, n are string lengths
- **Space Complexity:** O(m*n) for full table, can be optimized to O(min(m,n))

**ML Pipeline Application:** Detect duplicate patient records, match drug names, correct medical transcription errors.

**Healthcare Context:** Compare patient names like "Adebayo" vs "Adebayo" (typo detection) or drug names across different databases.

In [ ]:
# Healthcare examples: Patient name matching and drug similarity
examples = [
    ("Adebayo", "Adebayo"),  # Exact match
    ("Chukwuma", "Chukwuma"),  # Common Nigerian names
    ("Amoxicillin", "Amoxicilin"),  # Drug name typo
    ("Lagos General Hospital", "Lagos General Hospita"),  # Hospital name
]

print("🩺 Healthcare Text Similarity Analysis:")
print("String 1 → String 2 | Edit Distance | Similarity Score")
print("-" * 60)

for str1, str2 in examples:
    distance, dp_table = edit_distance(str1, str2)
    max_len = max(len(str1), len(str2))
    similarity = 1 - (distance / max_len)  # Normalized similarity
    
    print(f"{str1} → {str2} | {distance} | {similarity:.2f}")
    
    # Visualize first example
    if str1 == "Adebayo":
        visualize_edit_distance(str1, str2, dp_table)

# ML Application: Duplicate patient detection
patient_names = [
    "John Adebayo",
    "Jon Adebayo", 
    "John Adebayo",
    "Jane Adebayo",
    "John Adebayo Jr"
]

print("\n🔍 Patient Duplicate Detection:")
target = "John Adebayo"
print(f"Comparing all names to: {target}\n")

for name in patient_names:
    if name != target:
        distance, _ = edit_distance(target, name)
        similarity = 1 - (distance / max(len(target), len(name)))
        status = "POTENTIAL DUPLICATE" if similarity > 0.8 else "DIFFERENT"
        print(f"{name} | Distance: {distance} | Similarity: {similarity:.2f} | {status}")

**Healthcare Analysis:**

- **Name Matching:** Nigerian names like "Adebayo" vs "Adebayo" show edit distance patterns
- **Drug Safety:** Detecting medication name typos prevents prescription errors
- **Hospital Integration:** Matching facility names across different databases
- **Duplicate Detection:** Finding potential duplicate patient records

**ML Impact:** Enables accurate patient matching across fragmented Nigerian healthcare systems.

**Reflection Question:** How would edit distance help integrate patient records from Lagos and Abuja hospitals?

## Method 2: Knapsack Problem - Resource Allocation Optimization

**Healthcare Analogy:** Allocating limited medical supplies (beds, ventilators, medications) to maximize patient outcomes.

**ML Application:** Feature selection, resource-constrained optimization, budget allocation.

**Algorithm:** DP table where dp[i][w] represents maximum value using first i items with weight limit w.

In [ ]:
def knapsack_01(weights, values, capacity):
    """
    0/1 Knapsack: Select items to maximize value without exceeding capacity.
    Each item can be selected at most once.
    
    Parameters:
    weights: List of item weights
    values: List of item values  
    capacity: Maximum weight capacity
    
    Returns:
    max_value: Maximum achievable value
    dp_table: DP table for analysis
    selected_items: List of selected item indices
    """
    n = len(weights)
    
    # Initialize DP table: dp[i][w] = max value using first i items, capacity w
    dp = np.zeros((n+1, capacity+1), dtype=int)
    
    # Fill DP table
    for i in range(1, n+1):
        for w in range(capacity+1):
            # Option 1: Don't take current item
            dp[i][w] = dp[i-1][w]
            
            # Option 2: Take current item (if it fits)
            if weights[i-1] <= w:
                dp[i][w] = max(dp[i][w], dp[i-1][w - weights[i-1]] + values[i-1])
    
    # Reconstruct solution
    selected_items = []
    w = capacity
    for i in range(n, 0, -1):
        if dp[i][w] != dp[i-1][w]:  # Item was selected
            selected_items.append(i-1)
            w -= weights[i-1]
    
    return dp[n][capacity], dp, selected_items

def visualize_knapsack(dp_table, weights, values, capacity):
    """Visualize knapsack DP table"""
    fig, ax = plt.subplots(figsize=(12, 8))
    
    im = ax.imshow(dp_table, cmap='Greens')
    
    # Add value annotations for final row
    for w in range(capacity+1):
        text = ax.text(w, len(weights), dp_table[len(weights), w],
                     ha="center", va="center", color="black", fontsize=10)
    
    ax.set_xticks(np.arange(capacity+1))
    ax.set_yticks(np.arange(len(weights)+1))
    ax.set_xticklabels(range(capacity+1))
    ax.set_yticklabels([''] + [f'Item {i+1}\n(w={weights[i]}, v={values[i]})' 
                               for i in range(len(weights))])
    
    ax.set_xlabel('Capacity')
    ax.set_ylabel('Items Considered')
    ax.set_title('Knapsack DP Table\n(Darker = Higher Value)')
    
    plt.colorbar(im)
    plt.tight_layout()
    plt.show()

**Algorithm Analysis:**

- **DP Table**: dp[i][w] = maximum value using first i items with capacity w
- **Decisions**: For each item, choose maximum of (skip item, take item if fits)
- **Optimal Substructure**: Solution combines optimal solutions to subproblems
- **Time Complexity:** O(n*W) where n = items, W = capacity
- **Space Complexity:** O(n*W), can be optimized to O(W)

**ML Pipeline Application:** Feature selection with computational constraints, model ensemble optimization.

**Healthcare Context:** Allocate limited medical resources (ventilators, ICU beds) to maximize patient survival probability.

In [ ]:
# Healthcare resource allocation example
resources = {
    "Ventilators": {"weight": 50, "value": 95},  # kg, survival probability %
    "ICU Beds": {"weight": 30, "value": 85},
    "Oxygen Tanks": {"weight": 20, "value": 70},
    "Medications": {"weight": 15, "value": 60},
    "Monitoring Equipment": {"weight": 25, "value": 75}
}

weights = [info["weight"] for info in resources.values()]
values = [info["value"] for info in resources.values()]
resource_names = list(resources.keys())

# Transport capacity: 100kg (like a medical supply helicopter)
capacity = 100

max_value, dp_table, selected = knapsack_01(weights, values, capacity)

print("🏥 Healthcare Resource Allocation Optimization:")
print(f"Transport Capacity: {capacity}kg")
print(f"Maximum Survival Benefit: {max_value}%\n")

print("Selected Resources:")
total_weight = 0
for idx in selected:
    name = resource_names[idx]
    weight = weights[idx]
    value = values[idx]
    print(f"  ✓ {name}: {weight}kg, {value}% survival benefit")
    total_weight += weight

print(f"\nTotal Weight: {total_weight}kg (within {capacity}kg limit)")

# Visualize DP table
visualize_knapsack(dp_table, weights, values, capacity)

# ML Feature Selection Example
print("\n🤖 ML Feature Selection with Computational Constraints:")
features = {
    "Patient Age": {"complexity": 1, "importance": 8},
    "Blood Pressure": {"complexity": 2, "importance": 9},
    "Medical History": {"complexity": 5, "importance": 10},
    "Lab Results": {"complexity": 4, "importance": 9},
    "Demographics": {"complexity": 2, "importance": 6}
}

feature_weights = [info["complexity"] for info in features.values()]
feature_values = [info["importance"] for info in features.values()]
feature_names = list(features.keys())

# Computational budget: 10 complexity units
comp_budget = 10

max_importance, _, selected_features = knapsack_01(feature_weights, feature_values, comp_budget)

print(f"Computational Budget: {comp_budget} units")
print(f"Maximum Feature Importance: {max_importance}\n")

print("Selected Features:")
total_complexity = 0
for idx in selected_features:
    name = feature_names[idx]
    complexity = feature_weights[idx]
    importance = feature_values[idx]
    print(f"  ✓ {name}: {complexity} complexity, {importance} importance")
    total_complexity += complexity

print(f"\nTotal Complexity: {total_complexity} (within {comp_budget} budget)")

**Resource Optimization Analysis:**

- **Healthcare Allocation:** Selects most valuable medical resources within transport constraints
- **ML Feature Selection:** Chooses important features within computational limits
- **Trade-off Analysis:** Shows how DP balances competing objectives

**Nigerian Healthcare Impact:** Optimize resource distribution across hospitals with limited transportation capacity.

**Reflection Question:** How would knapsack optimization help allocate COVID-19 vaccines across Nigerian states?

## Method 3: Longest Common Subsequence - Pattern Discovery

**Healthcare Analogy:** Finding common genetic markers or symptom patterns across different diseases.

**ML Application:** Sequence alignment, plagiarism detection, version control diff algorithms.

**Algorithm:** DP table where dp[i][j] represents LCS length of first i chars of str1 and first j chars of str2.

In [ ]:
def longest_common_subsequence(str1, str2):
    """
    Find longest common subsequence between two strings.
    
    Parameters:
    str1, str2: Input strings
    
    Returns:
    lcs_length: Length of LCS
    lcs_string: Actual LCS string
    dp_table: DP table for analysis
    """
    m, n = len(str1), len(str2)
    
    # Initialize DP table
    dp = np.zeros((m+1, n+1), dtype=int)
    
    # Fill DP table
    for i in range(1, m+1):
        for j in range(1, n+1):
            if str1[i-1] == str2[j-1]:
                # Characters match - extend LCS
                dp[i][j] = dp[i-1][j-1] + 1
            else:
                # Characters differ - take maximum from left or above
                dp[i][j] = max(dp[i-1][j], dp[i][j-1])
    
    # Reconstruct LCS string
    lcs = []
    i, j = m, n
    while i > 0 and j > 0:
        if str1[i-1] == str2[j-1]:
            lcs.append(str1[i-1])
            i -= 1
            j -= 1
        elif dp[i-1][j] > dp[i][j-1]:
            i -= 1
        else:
            j -= 1
    
    lcs.reverse()
    lcs_string = ''.join(lcs)
    
    return dp[m][n], lcs_string, dp

def visualize_lcs(dp_table, str1, str2):
    """Visualize LCS DP table"""
    fig, ax = plt.subplots(figsize=(10, 8))
    
    im = ax.imshow(dp_table, cmap='Purples')
    
    # Add length annotations
    for i in range(len(str1)+1):
        for j in range(len(str2)+1):
            text = ax.text(j, i, dp_table[i, j],
                         ha="center", va="center", color="white", fontsize=10)
    
    ax.set_xticks(np.arange(len(str2)+1))
    ax.set_yticks(np.arange(len(str1)+1))
    ax.set_xticklabels([''] + list(str2))
    ax.set_yticklabels([''] + list(str1))
    
    ax.set_xlabel(f'String 2: "{str2}"')
    ax.set_ylabel(f'String 1: "{str1}"')
    ax.set_title('LCS DP Table\n(Darker = Longer Common Subsequence)')
    
    plt.colorbar(im)
    plt.tight_layout()
    plt.show()

**Algorithm Analysis:**

- **DP Table**: dp[i][j] = length of LCS between first i chars of str1 and first j chars of str2
- **Matching Characters**: Extend LCS when characters match
- **Non-matching**: Take maximum from skipping one character
- **Time Complexity:** O(m*n) for table filling, O(m+n) for reconstruction
- **Space Complexity:** O(m*n), optimizable to O(min(m,n))

**ML Pipeline Application:** Biological sequence alignment, text similarity analysis, version control.

**Healthcare Context:** Find common symptom patterns across different diseases or genetic markers.

In [ ]:
# Healthcare pattern discovery examples
examples = [
    ("MALARIA", "MALARIA"),  # Identical
    ("MALARIA", "MALNUTRITION"),  # Related diseases
    ("COVID19", "SARS"),  # Respiratory diseases
    ("DIABETES", "HYPERTENSION"),  # Chronic conditions
]

print("🧬 Disease Pattern Analysis (LCS):")
print("Disease 1 → Disease 2 | LCS Length | Common Pattern")
print("-" * 65)

for disease1, disease2 in examples:
    length, pattern, dp_table = longest_common_subsequence(disease1, disease2)
    similarity = length / min(len(disease1), len(disease2))
    
    print(f"{disease1} → {disease2} | {length} | '{pattern}' ({similarity:.2f})")
    
    # Visualize first example
    if disease1 == "MALARIA":
        visualize_lcs(dp_table, disease1, disease2)

# ML Application: Biological sequence alignment
print("\n🧬 DNA Sequence Pattern Discovery:")
dna_sequences = [
    "ATCGATCG",  # Reference sequence
    "ATCGTTTG",  # Similar sequence
    "GCTAGCTA",  # Different sequence
    "ATCGATCG",  # Identical to reference
]

reference = dna_sequences[0]
print(f"Reference DNA: {reference}\n")

for i, seq in enumerate(dna_sequences[1:], 1):
    length, pattern, _ = longest_common_subsequence(reference, seq)
    similarity = length / len(reference)
    match_type = "IDENTICAL" if similarity == 1.0 else "SIMILAR" if similarity > 0.5 else "DIFFERENT"
    
    print(f"Sequence {i}: {seq}")
    print(f"  LCS: '{pattern}' (length: {length})")
    print(f"  Similarity: {similarity:.2f} - {match_type}\n")

# Performance comparison: DP vs Recursive
def lcs_recursive(str1, str2, i=None, j=None):
    """Naive recursive LCS (exponential time)"""
    if i is None: i = len(str1)
    if j is None: j = len(str2)
    
    if i == 0 or j == 0:
        return 0
    
    if str1[i-1] == str2[j-1]:
        return lcs_recursive(str1, str2, i-1, j-1) + 1
    else:
        return max(lcs_recursive(str1, str2, i-1, j), 
                  lcs_recursive(str1, str2, i, j-1))

# Time comparison
test_str1 = "MALARIA"
test_str2 = "MALNUTRITION"

print("⚡ Performance Comparison:")

# DP approach
start = time.time()
dp_result = longest_common_subsequence(test_str1, test_str2)[0]
dp_time = time.time() - start

# Recursive approach (only for small strings!)
start = time.time()
recursive_result = lcs_recursive(test_str1, test_str2)
recursive_time = time.time() - start

print(f"DP Approach: {dp_result} (time: {dp_time:.6f}s)")
print(f"Recursive: {recursive_result} (time: {recursive_time:.6f}s)")
print(f"Speedup: {recursive_time/dp_time:.1f}x faster with DP!")

**Pattern Discovery Analysis:**

- **Disease Patterns:** Finds common substrings in disease names (like "MAL" in malaria/malnutrition)
- **DNA Sequences:** Identifies genetic similarities and differences
- **Performance:** DP dramatically outperforms naive recursive approach

**Healthcare Impact:** Enable genetic research and disease pattern recognition in Nigerian medical research.

**Reflection Question:** How would LCS help identify genetic markers common to different Nigerian ethnic groups?

## 🎯 Key Takeaways and Nigerian Healthcare Applications

**Algorithm Summary:**
- **Edit Distance:** Measures sequence similarity for text processing and duplicate detection
- **Knapsack:** Optimizes resource allocation under constraints
- **LCS:** Finds common patterns in sequences for biological and text analysis

**Healthcare Translation - Mark:**
Imagine building Nigeria's healthcare AI systems:
- **Edit Distance:** Match patient records across hospitals, detect drug name errors
- **Knapsack:** Allocate medical supplies efficiently during emergencies
- **LCS:** Discover genetic patterns in Nigerian population health studies

**Performance achieved:** DP algorithms enable efficient sequence optimization for large-scale ML applications!

**Reflection Questions:**
1. How would edit distance help integrate patient databases from different Nigerian hospitals?
2. What healthcare resources would you prioritize in a knapsack optimization for rural clinics?
3. How might LCS analysis reveal common genetic factors in Nigerian disease patterns?

**Next Steps:**
- Implement string algorithms for NLP preprocessing
- Apply DP to advanced tree structures
- Extend to sorting and searching optimizations

**🏆 Outstanding work, my student! You've mastered dynamic programming for sequence optimization.**